# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Highly Comparative Graph Analysis	Talk	complexnet	The 9th International Conference on Complex Networks and their Applications	2020-12-03	Madrid, Spain (Virtual)		
Overshooting behaviours in networks	Talk	coxic	COXIC	2020-12-14	London, UK		
Good practices in distributed and online learning	Panel discussion	ifest	iFest 2019	2019-08-12	Alexandria, US		
Tremor Analysis in Essential Tremor patients	Talk	iop	Complexity in the 21st Century. Institute of Physics	2019-07-18	London, UK		
Using time-series engagement data to predict student Performance	Talk	gmac	GMAC Leadership conference	2019-01-12	Fort Lauderdale, US		
Imperial College Business School	Talk	roundtable	Business School Round Table	2018-11-27	London, UK		
Learning analytics dashboard and student engagement behaviours	Talk	fome	FOME Oslo	2018-11-15	Oslo, Norway		
Predicting patient tremor response to TACS	Talk	cmph	Centre for Mathematical Precision Healthcare	2018-10-20	Lo

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Highly Comparative Graph Analysis,Talk,complexnet,The 9th International Conference on Complex Ne...,2020-12-03,"Madrid, Spain (Virtual)",NaN,NaN
1,Overshooting behaviours in networks,Talk,coxic,COXIC,2020-12-14,"London, UK",NaN,NaN
2,Good practices in distributed and online learning,Panel discussion,ifest,iFest 2019,2019-08-12,"Alexandria, US",NaN,NaN
3,Tremor Analysis in Essential Tremor patients,Talk,iop,Complexity in the 21st Century. Institute of P...,2019-07-18,"London, UK",NaN,NaN
4,Using time-series engagement data to predict s...,Talk,gmac,GMAC Leadership conference,2019-01-12,"Fort Lauderdale, US",NaN,NaN
5,Imperial College Business School,Talk,roundtable,Business School Round Table,2018-11-27,"London, UK",NaN,NaN
6,Learning analytics dashboard and student engag...,Talk,fome,FOME Oslo,2018-11-15,"Oslo, Norway",NaN,NaN
7,Predicting patient tremor response to TACS,Talk,cmph,Centre for Mathematical Precision Healthcare,2018-10-20,"London, UK",NaN,NaN
8,Predicting the effect of mutations on protein ...,Poster,tokyotech,Tokyo Tech-Imperial College workshop – Bioscie...,2016-11-05,"Tokyo, Japan",NaN,NaN
9,Markov Stability of Protein Structures,Poster,biophys,Conformational ensembles from experimental dat...,2017-08-02,"Berlin, Germany",NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2016-11-05-tokyotech.md  2018-11-27-roundtable.md  2020-12-03-complexnet.md
2017-08-02-biophys.md	 2019-01-12-gmac.md	   2020-12-14-coxic.md
2018-10-20-cmph.md	 2019-07-18-iop.md
2018-11-15-fome.md	 2019-08-12-ifest.md


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
